In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!pip install datasets
!pip install -U transformers datasets --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 60.9 MB/s eta 0:00:00


In [ ]:
hugging_face_token = ""
wnb_token = ""

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from datasets import Dataset
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix
from collections import Counter

###############################
# 🔧 Configuration et chargement du dataset
###############################

# Charger les données depuis le CSV
print("start reading files")
# 🔁 Modifier ce chemin si tu es dans Google Colab
data_folder = "./ciciot2023/versions/1"  # Exemple avec Colab

# 📂 Lire tous les fichiers CSV dans le dossier
csv_files = [f for f in os.listdir(data_folder) if f.endswith(".csv")]

# 📦 Charger et concaténer tous les fichiers
df = pd.concat([
    pd.read_csv(os.path.join(data_folder, file))
    for file in sorted(csv_files)
], ignore_index=True)

# ✅ Vérifier les premières lignes
print("✅ Données chargées :", df.shape)
print(df.head())
# Vérifier la distribution initiale des labels
class_counts = Counter(df["label"])
print("Distribution initiale des classes:", class_counts)

# Encodage des labels (on convertit en str pour garantir la cohérence et éviter les problèmes d'espaces)
df["label"] = df["label"].astype(str)

# Appliquer le label encoder pour transformer les labels en entiers
label_encoder = LabelEncoder()
df["label"] = label_encoder.fit_transform(df["label"])
print("Labels après encodage:", sorted(set(df["label"])))  # Affiche les indices uniques

# Séparation des données en train et test avec stratification
train_df, test_df = train_test_split(
    df,
    test_size=0.2,
    stratify=df["label"],
    random_state=42
)

# Conversion en datasets Hugging Face
dataset = {
    "train": Dataset.from_pandas(train_df),
    "test": Dataset.from_pandas(test_df)
}

# -----------------------------
# Vérification de la distribution après split
train_counts = Counter(dataset["train"]["label"])
print("Distribution des classes (train):", train_counts)

###############################
# 🔧 Prétraitement et création de prompts textuels
###############################

# Configuration du modèle et du tokenizer
model_name = "bert-base-uncased"  # Utilisation d'un BERT standard pour la classification
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(label_encoder.classes_)
)

# Fonction de prétraitement : on convertit les features (toutes les colonnes sauf "label") en texte
def preprocess_function(examples):
    # Récupérer les colonnes de features en excluant "label"
    feature_columns = [col for col in examples.keys() if col != "label"]
    texts = []
    # Pour chaque exemple du batch
    for i in range(len(examples["label"])):
        # Créer un texte en joignant : colonne:valeur (séparés par un espace)
        text = " ".join([f"{col}:{examples[col][i]}" for col in feature_columns])
        texts.append(text)
    tokenized = tokenizer(
        texts,
        padding="max_length",
        truncation=True,
        max_length=256  # Limite pour économiser des ressources sur le T4
    )
    tokenized["labels"] = examples["label"]
    return tokenized

# Appliquer le prétraitement sur les splits train et test
encoded_dataset = {
    "train": dataset["train"].map(preprocess_function, batched=True, batch_size=1000),
    "test": dataset["test"].map(preprocess_function, batched=True, batch_size=1000)
}

Distribution initiale des classes: Counter({'DDoS-ICMP_Flood': 36554, 'DDoS-UDP_Flood': 27626, 'DDoS-TCP_Flood': 23149, 'DDoS-PSHACK_Flood': 21210, 'DDoS-SYN_Flood': 20739, 'DDoS-RSTFINFlood': 20669, 'DDoS-SynonymousIP_Flood': 18189, 'DoS-UDP_Flood': 16957, 'DoS-TCP_Flood': 13630, 'DoS-SYN_Flood': 10275, 'BenignTraffic': 5600, 'Mirai-greeth_flood': 5016, 'Mirai-udpplain': 4661, 'Mirai-greip_flood': 3758, 'DDoS-ICMP_Fragmentation': 2377, 'MITM-ArpSpoofing': 1614, 'DDoS-ACK_Fragmentation': 1505, 'DDoS-UDP_Fragmentation': 1484, 'DNS_Spoofing': 925, 'Recon-HostDiscovery': 697, 'Recon-OSScan': 517, 'Recon-PortScan': 430, 'DoS-HTTP_Flood': 414, 'VulnerabilityScan': 210, 'DDoS-HTTP_Flood': 169, 'DDoS-SlowLoris': 106, 'DictionaryBruteForce': 63, 'SqlInjection': 31, 'BrowserHijacking': 30, 'CommandInjection': 28, 'Backdoor_Malware': 22, 'XSS': 18, 'Uploading_Attack': 8, 'Recon-PingSweep': 6})
Labels après encodage: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 2

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/190949 [00:00<?, ? examples/s]

Map:   0%|          | 0/47738 [00:00<?, ? examples/s]

In [17]:
training_args = TrainingArguments(
    output_dir="./results",
    eval_steps=500,
    logging_steps=100,
    do_eval=True,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,       # Or use max_steps=500
    weight_decay=0.01,
    logging_dir="./logs",
    fp16=True,                # Enable mixed precision
)



def compute_metrics(p):
    preds = p.predictions.argmax(-1)
    accuracy = (preds == p.label_ids).mean()
    return {"accuracy": accuracy}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["test"],
    compute_metrics=compute_metrics,
)

###############################
# 🔧 Entraînement
###############################
print("Début de l'entraînement ...")
trainer.train()
print("Fin de l'entraînement.")

###############################
# 🔎 Évaluation du modèle
###############################


Début de l'entraînement ...


Step,Training Loss
100,1.471900
200,1.235400
300,0.958600
400,0.744100
500,0.686500
600,0.684600
700,0.558600
800,0.572600
900,0.597300
1000,0.549900


Fin de l'entraînement.


In [18]:
results = trainer.evaluate()
print("Résultats de l'évaluation :", results)

# Obtenir les prédictions sur le dataset de test
predictions_output = trainer.predict(encoded_dataset["test"])
preds = predictions_output.predictions.argmax(-1)

# Calculer le rapport de classification
report = classification_report(encoded_dataset["test"]["labels"], preds, target_names=label_encoder.inverse_transform(range(len(label_encoder.classes_))), digits=4)
print("\n🔍 Rapport de Classification :")
print(report)

# Matrice de confusion
conf_matrix = confusion_matrix(encoded_dataset["test"]["labels"], preds)
print("\n📊 Matrice de Confusion :")
print(conf_matrix)

# Sauvegarder les résultats dans un fichier CSV
results_df = pd.DataFrame({
    "True Label": label_encoder.inverse_transform(encoded_dataset["test"]["labels"]),
    "Predicted Label": label_encoder.inverse_transform(preds)
})
results_df.to_csv("predictions.csv", index=False)
print("\n📁 Fichier 'predictions.csv' enregistré avec succès !")


Résultats de l'évaluation : {'eval_loss': 0.3352528214454651, 'eval_accuracy': 0.8704805396120491, 'eval_runtime': 179.301, 'eval_samples_per_second': 266.245, 'eval_steps_per_second': 16.642, 'epoch': 1.0}

🔍 Rapport de Classification :
                         precision    recall  f1-score   support

       Backdoor_Malware     0.0000    0.0000    0.0000         4
          BenignTraffic     0.7738    0.9500    0.8529      1120
       BrowserHijacking     0.0000    0.0000    0.0000         6
       CommandInjection     0.0000    0.0000    0.0000         6
 DDoS-ACK_Fragmentation     0.9481    0.9701    0.9589       301
        DDoS-HTTP_Flood     0.0000    0.0000    0.0000        34
        DDoS-ICMP_Flood     0.9862    0.9885    0.9874      7311
DDoS-ICMP_Fragmentation     0.8188    0.7705    0.7939       475
      DDoS-PSHACK_Flood     0.9995    0.9981    0.9988      4242
       DDoS-RSTFINFlood     1.0000    0.9995    0.9998      4134
         DDoS-SYN_Flood     0.6882    0.9405  

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [25]:
import pandas as pd
from sklearn.metrics import accuracy_score

# Load the predictions
df = pd.read_csv("/content/predictions.csv")

# Clean column names (remove whitespace)
df.columns = [col.strip() for col in df.columns]

# Compare predictions with true labels
accuracy = accuracy_score(df["True Label"], df["Predicted Label"])

print(f"✅ Accuracy: {accuracy:.4f} ({accuracy * 100:.2f}%)")


✅ Accuracy: 0.8705 (87.05%)
